In [1]:
import pandas as pd
import psycopg2

In [54]:
def db_connection(db_password, db_user, db_host, db_port, db_name):
    engine= psycopg2.connect(
        user= db_user,
        password= db_password,
        host= db_host,
        port= db_port,
        database= db_name
        
    )
    return engine

In [55]:
db_engine =db_connection(db_password='postgres',db_user='postgres', db_host='localhost',db_port=5432, db_name='bronze')

In [56]:
query= '''
    select * from retail_cleaned
'''
df = pd.read_sql(query,db_engine,index_col='Id').copy()
df.head()

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16640\983933411.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,db_engine,index_col='Id').copy()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
Id,,,,,,,,
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [5]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object

In [6]:
df.shape

(401604, 8)

In [59]:
from sqlalchemy import create_engine
def db_connection(user, password, host, port,db):
    engine= create_engine(
         f'postgresql://{user}:{password}@{host}:{port}/{db}'
    )
    engine.connect()
    return engine

In [60]:
db_engine =db_connection('postgres', 'postgres','localhost', 5432,'gold')

In [9]:
import pandas as pd
def Dimcustomer(cleaned_df, db_conn):
    DimCust = cleaned_df[['CustomerID', 'Country']].copy()
    DimCust['name'] = 'Customer' + DimCust['CustomerID'].astype(str)
    DimCust= DimCust.reset_index(drop=True)
    DimCust= DimCust.drop_duplicates('CustomerID').set_index('CustomerID')
    #DimCust.to_sql(name='dimcustomer', con=db_conn, if_exists='replace', index=True, index_label='CustomerID')
    return DimCust

dimCust = Dimcustomer(df, db_engine )
dimCust.head()

,Country,name
CustomerID,,
17850,United Kingdom,Customer17850
13047,United Kingdom,Customer13047
12583,France,Customer12583
13748,United Kingdom,Customer13748
15100,United Kingdom,Customer15100


In [10]:
dimCust.dtypes

Country    object
name       object
dtype: object

In [11]:
dimCust.shape

(4335, 2)

## prepar DimProduct


In [12]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
Id,,,,,,,,
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [13]:
def DimProduct(cleanedDF, engine):
    dimpro= cleanedDF[['StockCode', 'Description']]\
        .drop_duplicates(subset=['StockCode', 'Description'])\
        .reset_index(drop=True)
    dimpro['productKey']= range(1000, dimpro['StockCode'].count()+1000, 1)
    dimpro['name']= 'Product-'+dimpro['StockCode']
    #dimpro= dimpro.set_index('productKey')
    #dimpro.to_sql(name= 'dimproduct', con= engine, index=True, index_label='productKey')
    return dimpro

In [14]:
dimp= DimProduct(df, db_engine)
dimp.shape

(3912, 4)

In [15]:
dimp.head()

,StockCode,Description,productKey,name
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,1000,Product-85123A
1,71053,WHITE METAL LANTERN,1001,Product-71053
2,84406B,CREAM CUPID HEARTS COAT HANGER,1002,Product-84406B
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,1003,Product-84029G
4,84029E,RED WOOLLY HOTTIE WHITE HEART.,1004,Product-84029E


In [16]:
dimp.dtypes

StockCode      object
Description    object
productKey      int64
name           object
dtype: object

## DimDate
Date dimantion will be built on the top of invoice date column

In [17]:
invDateMax= df['InvoiceDate'].max()
invDateMax

Timestamp('2011-12-09 12:50:00')

In [18]:
invDateMin= df['InvoiceDate'].min()
invDateMin

Timestamp('1970-01-01 00:00:00')

In [19]:
def DimDate(cleaned_data):
    Dimdate= cleaned_data[['InvoiceDate']].drop_duplicates()
    Dimdate['DateKey'] = Dimdate['InvoiceDate']
    Dimdate['Date'] = Dimdate['InvoiceDate'].dt.strftime('%Y-%m-%d')
    Dimdate['Year']= Dimdate['InvoiceDate'].dt.strftime('%Y')
    Dimdate['MonthNo']= Dimdate['InvoiceDate'].dt.strftime('%m')
    Dimdate['MonthName']= Dimdate['InvoiceDate'].dt.strftime('%B')
    Dimdate['Day']= Dimdate['InvoiceDate'].dt.strftime('%d')
    Dimdate['quarter']= Dimdate['InvoiceDate'].dt.quarter
    Dimdate= Dimdate.reset_index(drop=True).set_index('DateKey')
    #Dimdate.to_sql(name= 'dimdate', con= db_engine, index=True, index_label='DateKey')
    return Dimdate
dd= DimDate(df)
dd.head()

,InvoiceDate,Date,Year,MonthNo,MonthName,Day,quarter
DateKey,,,,,,,
2010-12-01 08:26:00,2010-12-01 08:26:00,2010-12-01,2010,12,December,01,4
2010-12-01 08:28:00,2010-12-01 08:28:00,2010-12-01,2010,12,December,01,4
2010-12-01 08:34:00,2010-12-01 08:34:00,2010-12-01,2010,12,December,01,4
2010-12-01 08:35:00,2010-12-01 08:35:00,2010-12-01,2010,12,December,01,4
2010-12-01 08:45:00,2010-12-01 08:45:00,2010-12-01,2010,12,December,01,4


## Fact table
- I will join the denoramlized table with the dimantions to add the keys
- Then select the needed col in fact table
- Granularity is transactional

In [20]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
Id,,,,,,,,
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Join with DimProduct to add the product key

In [21]:
df.shape

(401604, 8)

In [63]:
def FactTable(cleaned_data, DimProduct, engine):
    fact =cleaned_data.merge(DimProduct, on=['StockCode','Description']).copy()
    fact= fact[['InvoiceNo','InvoiceDate','CustomerID', 'productKey','UnitPrice','Quantity']].set_index('InvoiceNo')
    fact= fact.rename(columns={
            'CustomerID':'CustomerKey',
            'InvoiceDate':'DateKey'
        })
    fact.to_sql(name='fact', con=engine, index=True, index_label='InvoiceNo')
    return fact
    


In [64]:
result =FactTable(df,dimp, db_engine)

In [65]:
result.head()

,DateKey,CustomerKey,productKey,UnitPrice,Quantity
InvoiceNo,,,,,
536365,2010-12-01 08:26:00,17850,1000,2.55,6
536373,2010-12-01 09:02:00,17850,1000,2.55,6
536375,2010-12-01 09:32:00,17850,1000,2.55,6
536390,2010-12-01 10:19:00,17511,1000,2.55,64
536394,2010-12-01 10:39:00,13408,1000,2.55,32
